In [44]:
import openai
from openai import OpenAI
import requests 
import os
from elasticsearch import Elasticsearch
from tqdm import tqdm

## Connecting to OpenAI

In [19]:
# I'll Use Gemini
model="gemini-2.5-flash-preview-05-20"
base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
api_key=os.getenv('GEMINI_API_KEY')

client = OpenAI(base_url=base_url, api_key=api_key)

In [20]:
# Test the client
messages = [{"role":"user" , "content":"is it late to join the course"}]
response = client.chat.completions.create(
    model=model,
    messages=messages,
    temperature=0.7,
)

In [21]:
response.choices[0].message.content

'It really depends on several factors! To give you the best advice, I\'d need to know:\n\n1.  **What kind of course is it?** (e.g., a university semester, a short online workshop, a self-paced certification, a coding bootcamp, a language class)\n2.  **When did it officially start?** (e.g., last week, a month ago, three months ago)\n3.  **What is the total duration of the course?** (e.g., 2 weeks, 8 weeks, 16 weeks, ongoing)\n4.  **How is the course structured?** (e.g., live lectures, pre-recorded videos, assignments, group projects, exams)\n5.  **Are there any official enrollment deadlines?** (Sometimes you simply can\'t join after a certain date.)\n\n**However, here\'s a general guide:**\n\n**It might be okay if:**\n\n*   **Only a small portion has passed:** If it\'s a long course (e.g., 16 weeks) and only 1-2 weeks have gone by.\n*   **Content is not heavily cumulative:** If each week\'s topic is relatively independent.\n*   **Materials are available:** You can access recordings of m

## Getting the data

In [4]:
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [5]:
len(documents)

948

## Indexing the data

In [41]:
elastic_client = Elasticsearch("http://localhost:9200")

In [43]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "module1-homework-questions"

elastic_client.indices.create(index=index_name , body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'module1-homework-questions'})

In [45]:
for doc in tqdm(documents):
    elastic_client.index(index=index_name , document=doc)

100%|████████████████████████████████████████████████████████████████████████████████| 948/948 [00:59<00:00, 16.04it/s]


## Searching in Documents

In [71]:
def elastic_search(query):
    
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }
    response = elastic_client.search(index=index_name,body=search_query)

    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit)

    return result_docs

In [72]:
query = "How do execute a command on a Kubernetes pod?"
elastic_search(query)

[{'_index': 'module1-homework-questions',
  '_id': '7BtsepcBQT9pif-cGpil',
  '_score': 44.50556,
  '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
   'section': '5. Deploying Machine Learning Models',
   'question': 'How do I debug a docker container?',
   'course': 'machine-learning-zoomcamp'}},
 {'_index': 'module1-homework-questions',
  '_id': 'extsepcBQT9pif-cPJmy',
  '_score': 35.433445,
  '_source': {'text': 'Deploy and Access the Kubernetes Dashboard\nLuke',
   'section': '10. Kubernetes and TensorFlow Serving',
   'question': 'Kubernetes-dashboard',
   'course': 'machine-learning-zoomcamp'}},
 {'_index': 'module1-homework-questions',
  '_id': 'DBtsepcBQT9pif-cIZnV',
  '_score': 33.7097

## Query Filtering

In [73]:
# Define the search query
search_results = elastic_client.search(
    index="module1-homework-questions",
    body={
        "query": {
            "bool": {
                "must": [
                    {
                        "multi_match": {
                            "query": "How do copy a file to a Docker container?",
                            "fields": ["question^4", "text"],
                            "type": "best_fields"
                        }
                    },
                    {
                        "term": {
                            "course": "machine-learning-zoomcamp"
                        }
                    }
                ]
            }
        },
        "size": 3  # Return only 3 results
    }
)

# Process and display results
print(f"Total matches: {search_results['hits']['total']['value']}")
print("Top 3 results:")
for i, hit in enumerate(search_results['hits']['hits'], 1):
    print(f"\nResult {i}:")
    print(f"Question: {hit['_source']['question']}")
    print(f"Course: {hit['_source']['course']}")
    print(f"Score: {hit['_score']:.2f}")

# Extract the 3rd question
if len(search_results['hits']['hits']) >= 3:
    third_question = search_results['hits']['hits'][2]['_source']['question']
    print(f"\nThe 3rd question returned is: '{third_question}'")
else:
    print("\nFewer than 3 results were returned")

Total matches: 340
Top 3 results:

Result 1:
Question: How do I debug a docker container?
Course: machine-learning-zoomcamp
Score: 74.31

Result 2:
Question: How do I copy files from my local machine to docker container?
Course: machine-learning-zoomcamp
Score: 67.62

Result 3:
Question: How do I copy files from a different folder into docker container’s working directory?
Course: machine-learning-zoomcamp
Score: 60.74

The 3rd question returned is: 'How do I copy files from a different folder into docker container’s working directory?'


## Building a prompt

In [92]:
def generat_prompt(question , search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
                """.strip()
    context = ""
    context_template = """
Q: {question}
A: {text}
            """.strip()
    
    for doc in search_results:
        doc = doc['_source']
        context = context + context_template.format(question=doc['question'] , text=doc['text']) + "\n\n"
    
    prompt = prompt_template.format(question=question, context=context).strip()
    print(prompt.strip())
    print("Length of the Prompt : ",len(prompt))
    
    return prompt

In [93]:
query = "How do copy a file to a Docker container?"

search_results = elastic_search(query)
prompt = generat_prompt(query,search_results)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do copy a file to a Docker container?

CONTEXT:
Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/contain

## Calculate Tokens Cost

In [95]:
# pip install tiktoken

In [96]:
import tiktoken

In [97]:
tokenizer = tiktoken.encoding_for_model("gpt-4o")


In [98]:
tokens = tokenizer.encode(prompt)
num_tokens = len(tokens)

# Output the result
print(f"Number of tokens: {num_tokens}")

Number of tokens: 320
